## 참고 : SVD 로 평점예측

https://data-science-hi.tistory.com/82

해당 SVD 에서는 아이템 별 평균을 구한 다음 사용자 별 평균을 빼주고, SVD 분해 이후 다시 사용자 별 평균을 더해준다.

In [1]:
import HA_ndcg

## 데이터 LOAD

In [2]:
import pandas as pd
import numpy as np
import math

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')

#컬럼명 수정
ratings.columns = ['userid','movieid','rating','timestamp']

#불필요한 컬럼 제거
ratings = ratings.drop(['timestamp'], axis=1)

#피벗테이블로 만들어서 사용자-아이템 행렬 확인하기
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])
rating_table

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 80000 row
user_train = pd.read_csv('../ml-100k_GRS/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]

# 20000 row
user_test = pd.read_csv('../ml-100k_GRS/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]

In [4]:
user_item_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_item_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

## 2. SVD 로 평점 예측

In [5]:
def R_filled_in(rating_table):
    for col in range(len(rating_table.columns)):
        col_update=[]
        # 컬럼의 평균을 구한다.
        col_num = [i for i in rating_table.iloc[:,col] if math.isnan(i)==False]
        col_mean = sum(col_num)/len(col_num)
        
        # NaN을 가진 행은 위에서 구한 평균 값으로 채워준다.
        col_update = [i if math.isnan(i)==False else col_mean for i in rating_table.iloc[:,col]]

        # 리스트로 만든 업데이트된 한 컬럼을 기존에 데이터 프레임 컬럼에 새로 입혀준다.
        rating_table.iloc[:,col] = col_update
        
    return rating_table
    

In [6]:
rating_R_filled = R_filled_in(rating_table)
rating_R_filled

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
2,4.000000,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,2.000000,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
3,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
4,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
5,4.000000,3.000000,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,3.878319,3.206107,3.033333,3.550239,3.302326,3.576923,3.798469,3.995434,5.000000,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
940,3.878319,3.206107,3.033333,2.000000,3.302326,3.576923,4.000000,5.000000,3.000000,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
941,5.000000,3.206107,3.033333,3.550239,3.302326,3.576923,4.000000,3.995434,3.896321,3.831461,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0


In [7]:
def R_norm(rating_R_filled):
    for row in range(len(rating_R_filled)):
        # 각 값에 평균을 빼준 값들을 넣을 리스트 공간 생성.
        row_update=[]

        # 행 평균 값
        row_mean=sum(rating_R_filled.iloc[row,:])/len(rating_R_filled.iloc[row,:])

        # 해당 행의 모든 컬럼 값에 행 평균 값을 뺀다.
        row_update= [i - row_mean for i in rating_R_filled.iloc[row,:]]

        rating_R_filled.iloc[row,:] = row_update

    return rating_R_filled, row_mean

rating_R_norm, row_mean = R_norm(rating_R_filled)

In [8]:
from sklearn.decomposition import TruncatedSVD

# TruncatedSVD를 사용해서 차원축소를 해보자. 여기서 k는 12로 두었다. n_iter는 랜덤 SVD 계산기의 반복횟수이다.
svd = TruncatedSVD(n_components=12, n_iter=5)

# 모델을 데이터에 맞추는 작업
svd.fit(np.array(rating_R_norm))

# 다음의 코드를 통해 각각 행렬 U, S, V^T 를 변수에 담아낼 수 있다.
U=svd.fit_transform(np.array(rating_R_norm))
Sigma=svd.explained_variance_ratio_
VT= svd.components_

In [9]:
# 시그마 행렬에 요소들에 루트를 씌워주는 작업.
Sigma_sqrt=[]
for i in range(len(Sigma)):
    tmp=[]
    tmp=[math.sqrt(s) for s in np.diag(Sigma)[i]]
    Sigma_sqrt.append(tmp)
        
Sigma_sqrt=np.array(Sigma_sqrt)    


# 행렬 곱
#pd.DataFrame(np.matmul(U, Sigma_sqrt))

# 행렬곱
#pd.DataFrame(np.matmul(Sigma_sqrt, VT))

In [10]:
ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT))
ratings_reduced

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.004868,0.002134,0.000288,0.009748,-0.002668,0.004245,0.013927,0.006796,0.009317,0.006026,...,-0.001130,0.005278,-0.001130,-0.007538,-0.001130,-0.013947,-0.001130,-0.007538,-0.001130,-0.001130
1,0.005154,0.000407,-0.000564,0.004215,0.001202,0.003851,0.005158,0.006465,0.007262,0.005255,...,-0.000578,0.006054,-0.000578,-0.007209,-0.000578,-0.013840,-0.000578,-0.007209,-0.000578,-0.000578
2,0.002060,0.000868,-0.001135,0.003501,-0.000103,0.002809,0.001670,0.007150,0.005215,0.004886,...,-0.000210,0.006623,-0.000210,-0.007044,-0.000210,-0.013877,-0.000210,-0.007044,-0.000210,-0.000210
3,0.008040,0.000737,0.000135,0.002624,0.002346,0.003899,0.007081,0.005369,0.005374,0.005610,...,-0.000652,0.006098,-0.000652,-0.007402,-0.000652,-0.014152,-0.000652,-0.007402,-0.000652,-0.000652
4,-0.002991,-0.001743,-0.001208,0.002281,-0.001769,0.005244,0.008404,0.001220,0.003944,0.007249,...,0.000031,0.007216,0.000031,-0.007153,0.000031,-0.014337,0.000031,-0.007153,0.000031,0.000031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.012754,0.003231,0.002185,0.003646,0.004291,0.002985,0.009535,0.007524,0.004496,0.004872,...,-0.001129,0.005429,-0.001129,-0.007687,-0.001129,-0.014245,-0.001129,-0.007687,-0.001129,-0.001129
939,0.004912,0.001200,0.000034,0.000170,0.002703,0.003224,0.003980,0.004395,0.002081,0.005002,...,-0.000399,0.006525,-0.000399,-0.007323,-0.000399,-0.014248,-0.000399,-0.007323,-0.000399,-0.000399
940,0.008293,0.001230,0.000265,0.003519,0.001703,0.003717,0.005879,0.006425,0.005626,0.005366,...,-0.000728,0.005944,-0.000728,-0.007400,-0.000728,-0.014072,-0.000728,-0.007400,-0.000728,-0.000728
941,0.011295,0.002456,-0.000347,0.004008,0.002567,0.002923,0.003430,0.007808,0.004675,0.003722,...,-0.000917,0.005566,-0.000917,-0.007400,-0.000917,-0.013883,-0.000917,-0.007400,-0.000917,-0.000917


해당 수치에 처음 뺐던 사용자 별 평균을 다시 더해준다.


In [11]:
def R_pred(ratings_reduced, row_mean):
    for row in range(len(ratings_reduced)):
        # 각 값에 평균을 더해 준 값들을 넣을 리스트 공간 생성.
        row_update=[]

        # 해당 행의 모든 컬럼 값에 행 평균 값을 뺀다.
        row_update= [i + row_mean for i in ratings_reduced.iloc[row,:]]

        ratings_reduced.iloc[row,:] = row_update

    return ratings_reduced

R = R_pred(ratings_reduced, row_mean)

R


,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.078950,3.076216,3.074370,3.083830,3.071414,3.078327,3.088009,3.080878,3.083399,3.080108,...,3.072952,3.079360,3.072952,3.066544,3.072952,3.060135,3.072952,3.066544,3.072952,3.072952
1,3.079236,3.074489,3.073518,3.078297,3.075284,3.077933,3.079240,3.080547,3.081344,3.079337,...,3.073504,3.080136,3.073504,3.066873,3.073504,3.060242,3.073504,3.066873,3.073504,3.073504
2,3.076142,3.074950,3.072947,3.077583,3.073979,3.076891,3.075752,3.081232,3.079297,3.078968,...,3.073872,3.080705,3.073872,3.067038,3.073872,3.060205,3.073872,3.067038,3.073872,3.073872
3,3.082122,3.074819,3.074217,3.076706,3.076428,3.077981,3.081163,3.079451,3.079456,3.079692,...,3.073430,3.080180,3.073430,3.066680,3.073430,3.059930,3.073430,3.066680,3.073430,3.073430
4,3.071091,3.072339,3.072874,3.076363,3.072313,3.079326,3.082486,3.075302,3.078026,3.081331,...,3.074113,3.081298,3.074113,3.066929,3.074113,3.059745,3.074113,3.066929,3.074113,3.074113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,3.086836,3.077313,3.076267,3.077728,3.078373,3.077067,3.083617,3.081606,3.078578,3.078954,...,3.072953,3.079511,3.072953,3.066395,3.072953,3.059837,3.072953,3.066395,3.072953,3.072953
939,3.078994,3.075282,3.074116,3.074252,3.076785,3.077306,3.078062,3.078477,3.076163,3.079084,...,3.073683,3.080607,3.073683,3.066759,3.073683,3.059834,3.073683,3.066759,3.073683,3.073683
940,3.082375,3.075312,3.074347,3.077601,3.075785,3.077800,3.079961,3.080507,3.079708,3.079448,...,3.073354,3.080026,3.073354,3.066682,3.073354,3.060011,3.073354,3.066682,3.073354,3.073354
941,3.085377,3.076538,3.073735,3.078090,3.076649,3.077005,3.077512,3.081890,3.078757,3.077804,...,3.073165,3.079648,3.073165,3.066682,3.073165,3.060199,3.073165,3.066682,3.073165,3.073165


In [16]:
import gambit

In [ ]:
game = nash.Game()

---

## 3. 클러스터링

In [12]:
HA_ndcg.groupRS_ndcg(3,R, user_item_train, user_item_test)

(3, 1682)
[array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.06547037, 3.06794455, 3.07094936, ..., 3.06755001, 3.07506235,
       3.07506235]), array([3.06547037, 3.06794455, 3.07094936, ..., 3.06755001, 3.07506235,
       3.07506235]), array([3.08976225, 3.08064177, 3.07639155, ..., 3.06630741, 3.07254613,
       3.07254613]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805, 3.0735884 ,
       3.0735884 ]), array([3.07924438, 3.0747568 , 3.07367699, ..., 3.06685805,

In [13]:
#cluster_num = range(2,51)

#for i in cluster_num:
#    HA_ndcg.groupRS_ndcg(i,R, user_item_train, user_item_test)

In [14]:
HA_ndcg.groupRS_ndcg(20,R, user_item_train, user_item_test)

(20, 1682)
[array([3.08065398, 3.07515958, 3.07392713, ..., 3.06681985, 3.0733202 ,
       3.0733202 ]), array([3.07780726, 3.07433932, 3.07344514, ..., 3.06692072, 3.07371299,
       3.07371299]), array([3.07780726, 3.07433932, 3.07344514, ..., 3.06692072, 3.07371299,
       3.07371299]), array([3.08042603, 3.07516404, 3.07391846, ..., 3.0667981 , 3.07350726,
       3.07350726]), array([3.06959081, 3.07069167, 3.07255181, ..., 3.06715013, 3.0744262 ,
       3.0744262 ]), array([3.06762396, 3.06909627, 3.0698809 , ..., 3.06759797, 3.07481755,
       3.07481755]), array([3.08781238, 3.07986566, 3.07518997, ..., 3.0664816 , 3.07237035,
       3.07237035]), array([3.08065398, 3.07515958, 3.07392713, ..., 3.06681985, 3.0733202 ,
       3.0733202 ]), array([3.08042603, 3.07516404, 3.07391846, ..., 3.0667981 , 3.07350726,
       3.07350726]), array([3.08065398, 3.07515958, 3.07392713, ..., 3.06681985, 3.0733202 ,
       3.0733202 ]), array([3.07780726, 3.07433932, 3.07344514, ..., 3.06692072

In [ ]:
km = KMeans(n_clusters=num, init='k-means++')
cluster = km.fit(total_matrix)
cluster_id = pd.DataFrame(cluster.labels_)                   # 모든  user에 클러스터 n 이 표시됨

cluster_id.index = total_matrix.index                        # userId 칼럼을 인덱스로 설정 1~943
cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터


In [15]:
HA_ndcg.groupRS_ndcg(20,R, user_item_train, user_item_test)

(20, 1682)
[array([3.08253275, 3.07563875, 3.07483184, ..., 3.06675355, 3.07291077,
       3.07291077]), array([3.0778019 , 3.0743344 , 3.07348144, ..., 3.06692241, 3.07370146,
       3.07370146]), array([3.0778019 , 3.0743344 , 3.07348144, ..., 3.06692241, 3.07370146,
       3.07370146]), array([3.08013892, 3.07503725, 3.07386575, ..., 3.06681075, 3.07353155,
       3.07353155]), array([3.0692211 , 3.07014562, 3.07163503, ..., 3.06730206, 3.07449637,
       3.07449637]), array([3.0692211 , 3.07014562, 3.07163503, ..., 3.06730206, 3.07449637,
       3.07449637]), array([3.08986099, 3.08075425, 3.07634655, ..., 3.06633412, 3.07217085,
       3.07217085]), array([3.08341085, 3.07668744, 3.07393314, ..., 3.06671622, 3.07318705,
       3.07318705]), array([3.08013892, 3.07503725, 3.07386575, ..., 3.06681075, 3.07353155,
       3.07353155]), array([3.08253275, 3.07563875, 3.07483184, ..., 3.06675355, 3.07291077,
       3.07291077]), array([3.0778019 , 3.0743344 , 3.07348144, ..., 3.06692241